In [1]:
%cd ..

/home/teo/userdata/git_libraries/Yeominrak


In [2]:
from pathlib import Path
import music21
from music21 import converter, stream, note as m21_note 
from typing import List, Set, Dict, Tuple
import torch
from collections import defaultdict, Counter
import copy
import numpy as np

from yeominrak_processing import AlignedScore, SamplingScore, pack_collate, ShiftedAlignedScore, TestScore, Gnote
from model_zoo import Seq2seq, Converter, AttentionSeq2seq, QkvAttnSeq2seq, get_emb_total_size
import random as random
from loss import nll_loss
from trainer import Trainer
from decode import MidiDecoder
from torch.nn.utils.rnn import pack_sequence, PackedSequence, pad_packed_sequence, pack_padded_sequence
import torch
from torch.utils.data import Dataset, DataLoader
from omegaconf import OmegaConf
from metric import make_dynamic_template, convert_note_to_sampling, get_similarity, get_correspondence
from constants import get_dynamic

In [3]:
config = OmegaConf.load('yamls/baseline.yaml')
config = get_emb_total_size(config)
device = 'cpu'
train_dataset = ShiftedAlignedScore(xml_path='0_edited.musicxml', is_valid=False, min_meas=4, max_meas=4, slice_measure_num=4)
val_dataset = ShiftedAlignedScore(xml_path='0_edited.musicxml', is_valid= True, slice_measure_num=4)


In [5]:
class BlockDataset:
  def __init__(self, dataset: ShiftedAlignedScore):
    self.data = dataset
    return
  
  def __len__(self):
    return 
  
  def __getitem__(self, idx):

    return
  

block_set = BlockDataset(train_dataset)


In [8]:
BLOCK_ANCHOR = [
  [0, 1.5, 2.5, 4],
  [0, 3, 6, 8],
  [0, 3, 6, 8, 10],
  [0, 3, 6, 8, 10],
  [0, 3, 6, 8, 10],
  [0, 3, 6, 8, 10],
  [0, 3, 5, 8, 10],
  [0, 3, 5, 8, 10]
]

def get_block_from_measures(measures: List[Gnote], measure_idx, block_idx, part_idx):
  anchor_point = BLOCK_ANCHOR[part_idx]
  block_start = anchor_point[block_idx]
  block_end = anchor_point[block_idx+1]

  selected_measure = measures[measure_idx]
  selected_notes = []
  for note in selected_measure:
    if note.measure_offset >= block_start and note.measure_offset < block_end:
      selected_notes.append(note)
  return selected_notes

def get_continuous_block(measures: List[Gnote], measure_idx, block_idx, part_idx):
  anchor_point = BLOCK_ANCHOR[part_idx]

  current_block = get_block_from_measures(measures, measure_idx, block_idx, part_idx)
  prev_block_idx = block_idx - 1
  prev_block_measure_idx = measure_idx
  if prev_block_idx < 0:
    prev_block_measure_idx -= 1
    prev_block_idx = len(anchor_point) - 2
  prev_block = get_block_from_measures(measures, prev_block_measure_idx, prev_block_idx, part_idx)

  next_block_idx = block_idx + 1
  next_block_measure_idx = measure_idx
  if next_block_idx >= len(anchor_point) - 1:
    next_block_measure_idx += 1
    next_block_idx = 0
  next_block = get_block_from_measures(measures, next_block_measure_idx, next_block_idx, part_idx)

  return prev_block, current_block, next_block


get_continuous_block(block_set.data.parts[7].measures, 5, 2, 1)

([Gnote: 46.0, 1.0, 3.0, middle,
  Gnote: 34.0, 1.0, 4.0, weak,
  Gnote: 46.0, 1.0, 5.0, strong],
 [Gnote: 48.0, 1.0, 6.0, weak, Gnote: 46.0, 1.0, 7.0, middle],
 [Gnote: 41.0, 1.0, 0.0, strong,
  Gnote: 46.0, 0.5, 1.0, weak,
  Gnote: 41.0, 0.5, 1.5, weak,
  Gnote: 39.0, 1.0, 2.0, weak])

In [7]:
part_measure_with_errors = []

for part_idx, part in enumerate(train_dataset.parts):
  for measuer_idx, measure in enumerate(part.measures):
    if measure == []:
      continue
    duration = sum([note.duration for note in measure])
    if part_idx in [0] and abs(duration-4) > 0.1 \
    or part_idx in [1] and abs(duration-8) > 0.1 \
    or part_idx in [2,3,4,5,6,7] and abs(duration-10) > 0.1:
      print(part_idx, measuer_idx, duration)
      part_measure_with_errors.append((part_idx+1, measuer_idx+1))

print(part_measure_with_errors)


2 42 9.0
2 49 9.0
2 56 9.0
2 108 9.0
3 42 9.0
3 49 9.0
3 56 9.0
3 84 9.0
3 108 9.0
4 42 4.0
4 49 9.0
4 54 6.0
4 64 2.0
5 13 6.0
5 25 6.0
5 49 9.0
5 51 7.0
5 89 6.0
5 114 9.0
[(3, 43), (3, 50), (3, 57), (3, 109), (4, 43), (4, 50), (4, 57), (4, 85), (4, 109), (5, 43), (5, 50), (5, 55), (5, 65), (6, 14), (6, 26), (6, 50), (6, 52), (6, 90), (6, 115)]


In [24]:
from collections import Counter
for part_idx, part in enumerate(train_dataset.parts):
  notes = [note.pitch for measure in part.measures for note in measure] 
  counter = Counter(notes)
  print(part_idx, counter)

0 Counter({53.0: 37, 48.0: 36, 46.0: 36, 44.0: 34, 51.0: 29, 41.0: 9, 39.0: 7, 56.0: 6})
1 Counter({46.0: 112, 48.0: 103, 53.0: 92, 44.0: 85, 51.0: 83, 56.0: 44, 41.0: 27, 39.0: 19, 58.0: 7})
2 Counter({51.0: 181, 48.0: 174, 46.0: 161, 53.0: 158, 56.0: 92, 44.0: 89, 41.0: 36, 58.0: 26, 39.0: 17, 60.0: 6, 49.0: 4})
3 Counter({51.0: 174, 48.0: 168, 46.0: 168, 53.0: 141, 44.0: 98, 56.0: 75, 41.0: 51, 39.0: 21, 58.0: 20, 60.0: 5, 49.0: 4, 63.0: 1})
4 Counter({51.0: 79, 46.0: 76, 48.0: 67, 53.0: 52, 44.0: 51, 39.0: 35, 56.0: 33, 41.0: 33, 58.0: 4, 49.0: 2, 60.0: 1})
5 Counter({51.0: 134, 46.0: 123, 53.0: 109, 48.0: 108, 56.0: 64, 44.0: 64, 39.0: 49, 41.0: 37, 58.0: 18, 60.0: 4, 55.0: 2, 49.0: 2, 63.0: 1})
6 Counter({51.0: 294, 46.0: 282, 48.0: 231, 53.0: 196, 56.0: 162, 44.0: 154, 39.0: 88, 41.0: 62, 58.0: 41, 60.0: 9, 49.0: 8, 34.0: 5, 63.0: 3})
7 Counter({46.0: 298, 51.0: 286, 48.0: 222, 53.0: 196, 44.0: 148, 56.0: 144, 39.0: 97, 41.0: 65, 58.0: 39, 60.0: 11, 49.0: 9, 34.0: 7, 63.0: 7})


In [20]:
with open('part_measure_with_errors.txt', 'w') as file:
  for item in part_measure_with_errors:
    file.write(f"파트: {item[0]}, 마디: {item[1]}\n")


In [19]:
len(part_measure_with_errors)

23

In [9]:
train_dataset.parts[0].measures

[[Gnote: 51.0, 4.0, 0.0, strong],
 [Gnote: 51.0, 4.0, 0.0, strong],
 [Gnote: 48.0, 4.0, 0.0, strong],
 [Gnote: 46.0, 2.5, 0.0, strong, Gnote: 48.0, 1.5, 2.5, strong],
 [Gnote: 53.0, 4.0, 0.0, strong],
 [Gnote: 53.0, 4.0, 0.0, strong],
 [Gnote: 53.0, 4.0, 0.0, strong],
 [Gnote: 53.0, 4.0, 0.0, strong],
 [Gnote: 51.0, 4.0, 0.0, strong],
 [Gnote: 51.0, 4.0, 0.0, strong],
 [Gnote: 53.0, 4.0, 0.0, strong],
 [Gnote: 53.0, 2.5, 0.0, strong, Gnote: 51.0, 1.5, 2.5, strong],
 [Gnote: 48.0, 4.0, 0.0, strong],
 [Gnote: 46.0, 2.5, 0.0, strong, Gnote: 44.0, 1.5, 2.5, strong],
 [Gnote: 46.0, 4.0, 0.0, strong],
 [Gnote: 48.0, 2.5, 0.0, strong, Gnote: 46.0, 1.5, 2.5, strong],
 [Gnote: 44.0, 4.0, 0.0, strong],
 [Gnote: 44.0, 2.5, 0.0, strong,
  Gnote: 46.0, 1.0, 2.5, strong,
  Gnote: 44.0, 0.5, 3.5, weak],
 [Gnote: 41.0, 4.0, 0.0, strong],
 [Gnote: 39.0, 2.5, 0.0, strong, Gnote: 41.0, 1.5, 2.5, strong],
 [Gnote: 44.0, 2.5, 0.0, strong, Gnote: 46.0, 1.5, 2.5, strong],
 [Gnote: 48.0, 4.0, 0.0, strong],
 [